In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load your dataset
# Make sure to adjust the read_csv path to the actual CSV file with market data
# Example: your_data.csv should contain columns 'Open', 'Low', 'High'
df = pd.read_csv('/content/TISC Historical Data (3).csv')
# Inspect the first few rows
print(df.head())

# Prepare features and target variable
print(df.info())

         Date  Price   Open   High    Low     Vol. Change %
0  08-02-2021  70.29  69.40  70.49  68.27  214.62M    2.61%
1  05-02-2021  68.50  65.69  69.06  65.14  291.01M    4.44%
2  04-02-2021  65.60  65.40  66.35  63.86  147.65M    0.43%
3  03-02-2021  65.32  64.49  66.43  63.72  148.46M    1.79%
4  02-02-2021  64.17  64.30  64.80  62.41  180.49M    0.87%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      5000 non-null   object 
 1   Price     5000 non-null   float64
 2   Open      5000 non-null   float64
 3   High      5000 non-null   float64
 4   Low       5000 non-null   float64
 5   Vol.      5000 non-null   object 
 6   Change %  5000 non-null   object 
dtypes: float64(4), object(3)
memory usage: 273.6+ KB
None


In [6]:
X = df[['Open', 'Low']].values
y = df['High'].values

In [14]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the data
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()

In [8]:
model = Sequential([
    Dense(356, activation='relu', input_dim=2),
    Dense(832, activation='relu'),
    Dense(741, activation='relu'),
    Dense(625, activation='relu'),
    Dense(529, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

# Train the model using training data
model.fit(X_train_scaled, y_train_scaled, epochs=100, batch_size=100)

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - loss: 0.1299 - mae: 0.1299
Epoch 2/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0324 - mae: 0.0324
Epoch 3/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 0.0232 - mae: 0.0232
Epoch 4/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: 0.0125 - mae: 0.0125
Epoch 5/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.0115 - mae: 0.0115
Epoch 6/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - loss: 0.0141 - mae: 0.0141
Epoch 7/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.0149 - mae: 0.0149
Epoch 8/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.0127 - mae: 0.0127
Epoch 9/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - loss: 0.0115 - mae: 0.0115
Epoch 10/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 0.0130 - mae: 0.0130
Epoch 11/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 0.0212 - mae: 0.0212
Epoch 12/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 0.0108 - mae: 0.0108
Epoch 13/100
40/40 ━━━━━━━━━━━━━━━━━━

In [9]:
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
y_pred_scaled = model.predict(X_test_scaled)
y_pred = y_scaler.inverse_transform(y_pred_scaled)

# Evaluation metrics
print("mse:", mean_squared_error(y_test, y_pred))
print("accuracy r2:", r2_score(y_test, y_pred))
print("mae:", mean_absolute_error(y_test, y_pred))

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step
mse: 0.5560686327796219
accuracy r2: 0.9983397756469267
mae: 0.4515028167819977


In [16]:
def predict_high(open_price, low_price):
    input_data = np.array([[open_price, low_price]])
    input_scaled = X_scaler.transform(input_data)  # Scale input
    predicted_high_scaled = model.predict(input_scaled)  # Make prediction
    predicted_high = y_scaler.inverse_transform(predicted_high_scaled.reshape(-1, 1))  # Inverse scale
    return predicted_high[0][0]

In [24]:
try:
    open_price = float(input("Enter the Open price: "))
    low_price = float(input("Enter the Low price: "))
    predicted_high = predict_high(open_price, low_price)
    print(f'Predicted High price: {predicted_high}')
except ValueError:
    print("Please enter valid numerical values for Open and Low prices.")

Enter the Open price: 137.00
Enter the Low price: 134.31
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted High price: 139.091796875


In [18]:
model.save('mytatasteel.keras')

In [23]:
from tensorflow import keras

# Load the saved model
loaded_model = keras.models.load_model('mytatasteel.keras')

# Now you can use the loaded_model with your predict_high function:
def predict_high(open_price, low_price):
    input_data = np.array([[open_price, low_price]])
    input_scaled = X_scaler.transform(input_data)
    predicted_high_scaled = loaded_model.predict(input_scaled)
    predicted_high = y_scaler.inverse_transform(predicted_high_scaled.reshape(-1, 1))
    return predicted_high[0][0]

# Get predictions
try:
    open_price = float(input("Enter the Open price: "))
    low_price = float(input("Enter the Low price: "))
    predicted_high = predict_high(open_price, low_price)
    print(f'Predicted High price: {predicted_high}')
except ValueError:
    print("Please enter valid numerical values for Open and Low prices.")

Enter the Open price: 138.81
Enter the Low price: 136.66


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Predicted High price: 141.1476287841797


In [25]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 356)                 │           1,068 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 832)                 │         297,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 741)                 │         617,253 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 625)                 │         463,750 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 529)                 │         331,154 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 128)                 │          67,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 8)                   │             520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,360,624 (20.45 MB)

 Trainable params: 1,786,874 (6.82 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,573,750 (13.63 MB)

In [28]:
y_test[0]

np.float64(34.88)

In [29]:
y_pred[0]

array([34.23283], dtype=float32)